In [1]:
import gym
gym.logger.set_level(40) # suppress warnings (please remove if gives error)
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
#env = gym.make('CartPole-v0')
env = gym.make('Bowling-ram-v0')
env.seed(0)
print('observation space:', env.observation_space)
print('Atlantis-ram-v0:', env.action_space)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):
    #def __init__(self, s_size=4, h_size=16, a_size=2):
    def __init__(self, s_size=128, a_size=6):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_size, 16)
        #self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(16, 32)
        #self.bn2 = nn.BatchNorm1d(1024)
        self.fc3 = nn.Linear(32, 16)
        #self.bn3 = nn.BatchNorm1d(256)
        
        self.fc4 = nn.Linear(16, a_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc1(x))
        #x = self.fc2(x)
        x = self.fc4(x)
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

observation space: Box(128,)
Atlantis-ram-v0: Discrete(6)


In [14]:
policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-2)

def reinforce(n_episodes=1000, max_t=1000, gamma=1.0, print_every=2):
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))
        
        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])
        
        policy_loss = []
        for log_prob in saved_log_probs:
            policy_loss.append(-log_prob * R)
        policy_loss = torch.cat(policy_loss).sum()
        
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
        #if i_episode % print_every == 0:
         #   torch.save(policy.state_dict(), 'iter%d.pth' % i_episode)
          #  Print()
    return scores
    
scores = reinforce()#env = gym.make('CartPole-v0')


Episode 2	Average Score: 0.00
Episode 4	Average Score: 0.00
Episode 6	Average Score: 0.00
Episode 8	Average Score: 0.00
Episode 10	Average Score: 0.00
Episode 12	Average Score: 0.00
Episode 14	Average Score: 0.00
Episode 16	Average Score: 0.00
Episode 18	Average Score: 0.00
Episode 20	Average Score: 0.00
Episode 22	Average Score: 0.00
Episode 24	Average Score: 0.00
Episode 26	Average Score: 0.00
Episode 28	Average Score: 0.00
Episode 30	Average Score: 0.00
Episode 32	Average Score: 0.00
Episode 34	Average Score: 0.00
Episode 36	Average Score: 0.00
Episode 38	Average Score: 0.00
Episode 40	Average Score: 0.00
Episode 42	Average Score: 0.00
Episode 44	Average Score: 0.00
Episode 46	Average Score: 0.00


KeyboardInterrupt: 

In [15]:
#env = gym.make('CartPole-v0')
import time
env = gym.make('Bowling-ram-v0')

state = env.reset()
for t in range(1000):
    action, _ = policy.act(state)
    env.render()
    state, reward, done, _ = env.step(action)
    time.sleep(0.01)
    if done:
        break 

env.close()